In [1]:
import pandas as pd
from openpyxl import load_workbook

In [2]:
filename = 'anamnese_sentencas.xlsx'

wb = load_workbook(filename)

In [3]:
def update_state(state, row):
    if row["Papel"].strip() == state["Papel"]:
        state["Contador"] += 1
    else:
        if row["Papel"].strip() == "Médico":
            state["Papel"] = "Médico"
            state["Médico"] += 1
        else:
            state["Papel"] = "Paciente"
            state["Paciente"] += 1
        state["Contador"] = 1
    return state

def get_id(state, sheet_prefix):
    persona_initial = state["Papel"][0].lower()

    return f"{sheet_prefix}_{persona_initial}{state[state['Papel']]:03d}_{state['Contador']:02d}"

In [4]:
for sheet_name in wb.sheetnames:
    df = pd.read_excel(filename, sheet_name=sheet_name)
    ws = wb[sheet_name]

    sheet_prefix = df.at[0, "ID"].split('_')[0]
    state = {
        "Papel": "Nenhum papel",
        "Médico": 0,
        "Paciente": 0,
        "Contador": 1,
    }

    for index, row in df.iterrows():
        state = update_state(state, row)
        id_col_index = df.columns.get_loc("ID") + 1
        id_row_index = index + 2
        ws.cell(row=id_row_index, column=id_col_index, value=get_id(state, sheet_prefix))
        
wb.save("anamnese_sentencas_corrigidas.xlsx")